# Tarea 5

In [101]:
# librerias
import os, mlflow
from dotenv import load_dotenv
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [102]:
load_dotenv(override=True)
EXPERIMENT_NAME = "/Users/sarahbeltrang@gmail.com/nyc-taxi-experiments"
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/86265904456551', creation_time=1761101109923, experiment_id='86265904456551', last_update_time=1761662610718, lifecycle_stage='active', name='/Users/sarahbeltrang@gmail.com/nyc-taxi-experiments', tags={'mlflow.experiment.sourceName': '/Users/sarahbeltrang@gmail.com/nyc-taxi-experiments',
 'mlflow.experimentKind': 'custom_model_development',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'sarahbeltrang@gmail.com',
 'mlflow.ownerId': '76150295292949'}>

In [103]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [104]:
df_train = read_dataframe("C:/Users/sarah/apps/sem_5/pcd/nyc-taxi-predictions-2025/data/green_tripdata_2025-01.parquet")
df_val = read_dataframe("C:/Users/sarah/apps/sem_5/pcd/nyc-taxi-predictions-2025/data/green_tripdata_2025-02.parquet")
df_test = read_dataframe("C:/Users/sarah/apps/sem_5/pcd/nyc-taxi-predictions-2025/data/green_tripdata_2025-03.parquet")

In [ ]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [142]:
from sklearn.feature_extraction import DictVectorizer

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# --- Ajustar DictVectorizer con training ---
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# --- Transformar validation con el mismo dv ---
X_val = preprocess(df_val, dv)

# --- Transformar test con el mismo dv ---
test_dicts = df_test[categorical + numerical].to_dict(orient='records')
X_test = dv.transform(test_dicts)
X_test = preprocess(df_test, dv)

# --- Opcional: convertir a DataFrame con nombres de columnas ---
X_train_df = pd.DataFrame(X_train.toarray(), columns=dv.get_feature_names_out())
X_val_df = pd.DataFrame(X_val.toarray(), columns=dv.get_feature_names_out())
X_test_df = pd.DataFrame(X_test.toarray(), columns=dv.get_feature_names_out())


In [143]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values
y_test = df_test[target].values

In [144]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [145]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import mlflow.sklearn

In [146]:
# Transformación de training
train_dicts = df_train[['PULocationID','DOLocationID','trip_distance']].copy()
train_dicts['PU_DO'] = train_dicts['PULocationID'] + '_' + train_dicts['DOLocationID']
train_dicts = train_dicts[['PU_DO','trip_distance']].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
y_train = df_train.duration.values

# Transformación de validación usando el mismo dv
val_dicts = df_val[['PULocationID','DOLocationID','trip_distance']].copy()
val_dicts['PU_DO'] = val_dicts['PULocationID'] + '_' + val_dicts['DOLocationID']
val_dicts = val_dicts[['PU_DO','trip_distance']].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_val.duration.values

# Transformación de test usando el mismo dv
test_dicts = df_test[['PULocationID','DOLocationID','trip_distance']].copy()
test_dicts['PU_DO'] = test_dicts['PULocationID'] + '_' + test_dicts['DOLocationID']
test_dicts = test_dicts[['PU_DO','trip_distance']].to_dict(orient='records')
X_test = dv.transform(test_dicts)
y_test = df_test.duration.values


In [148]:
print("X_train shape:", X_train.shape)
print("X_val shape:  ", X_val.shape)
print("X_test shape: ", X_test.shape)


X_train shape: (46307, 4159)
X_val shape:   (44218, 4159)
X_test shape:  (48336, 4159)


## Experiments

In [147]:
import math
import pickle
import pathlib
import optuna
import mlflow
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature


### GradientBoosting

In [63]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna - Gradient Boosting
# ------------------------------------------------------------
def objective_gb(trial: optuna.trial.Trial):
    # Hiperparámetros muestreados por Optuna en CADA trial
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "random_state": 89,
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "gradient_boosting")
        mlflow.log_params(params)

        # Entrenamiento del modelo
        model = GradientBoostingRegressor(**params)
        model.fit(X_train, y_train)

        # Predicción y cálculo del RMSE
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow
        mlflow.sklearn.log_model(
            model,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # ------------------------------------------------------------
    # RETURN para que Optuna minimice la métrica
    # ------------------------------------------------------------
    return rmse


### Flujo de busqueda - GB

In [64]:
# ============================================================
# FLUJO DE BÚSQUEDA Y ENTRENAMIENTO FINAL - GRADIENT BOOSTING
# ============================================================
mlflow.sklearn.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler
#    - direction="minimize" porque queremos minimizar el RMSE
# ------------------------------------------------------------
sampler = TPESampler(seed=89)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = 10)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros
#    - Abrimos un run "padre" para agrupar toda la búsqueda
# ------------------------------------------------------------
with mlflow.start_run(run_name="Gradient Boosting Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective_gb, n_trials=10)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    best_params["random_state"] = 89
    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "gradient_boosting",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # Entrenar un modelo FINAL con los mejores hiperparámetros
    # --------------------------------------------------------
    model = GradientBoostingRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # Guardar artefactos adicionales (preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # --------------------------------------------------------
    # Guardar modelo final con signature
    # --------------------------------------------------------
    signature = infer_signature(X_val, y_pred)
    mlflow.sklearn.log_model(
        model,
        name="model",
        input_example=X_val[:5],
        signature=signature
    )


[I 2025-10-28 08:31:09,186] A new study created in memory with name: no-name-fdf35d27-074a-4abe-be81-bcc0f0609568


2025/10/28 08:31:49 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 08:31:53,423] Trial 0 finished with value: 5.601293580469192 and parameters: {'n_estimators': 275, 'max_depth': 4, 'learning_rate': 0.024057247038385035, 'subsample': 0.5484608547195821, 'min_samples_split': 7, 'min_samples_leaf': 1}. Best is trial 0 with value: 5.601293580469192.


🏃 View run dapper-duck-49 at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/5b6e5bc6c72f41959138fec9faa6a08c
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


2025/10/28 08:32:31 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 08:32:35,676] Trial 1 finished with value: 5.787041863552104 and parameters: {'n_estimators': 118, 'max_depth': 6, 'learning_rate': 0.014045078672943832, 'subsample': 0.7084819461637473, 'min_samples_split': 10, 'min_samples_leaf': 4}. Best is trial 0 with value: 5.601293580469192.


🏃 View run masked-koi-472 at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/606fa7c25d9a45d99348c83a7cdb146c
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


...

🏃 View run placid-loon-382 at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/48e8894e2f3742929e9a40027f93fdad
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


2025/10/28 08:38:11 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 08:38:15,853] Trial 9 finished with value: 5.5642126270182635 and parameters: {'n_estimators': 290, 'max_depth': 7, 'learning_rate': 0.012130083137673756, 'subsample': 0.6299145441235463, 'min_samples_split': 10, 'min_samples_leaf': 1}. Best is trial 8 with value: 5.437591039019735.


🏃 View run calm-bear-238 at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/1b0f4edf203847feb101421f81369a5a
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


2025/10/28 08:38:46 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run Gradient Boosting Hyperparameter Optimization (Optuna) at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/0ead7f85fdf54fc3bdecf821ade07049
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


### RandomForest

In [65]:
def objective_rf(trial: optuna.trial.Trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 4, 50),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "random_state": 89,
    }

    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "random_forest")
        mlflow.log_params(params)

        model = RandomForestRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

        signature = infer_signature(X_val, y_pred)
        mlflow.sklearn.log_model(
            model,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # ------------------------------------------------------------
    # RETURN para que Optuna minimice la métrica
    # ------------------------------------------------------------
    return rmse

### Flujo de Busqueda - RF

In [66]:
# ============================================================
# FLUJO DE BÚSQUEDA Y ENTRENAMIENTO FINAL - RANDOM FOREST
# ============================================================
mlflow.sklearn.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler
#    - direction="minimize" porque queremos minimizar el RMSE
# ------------------------------------------------------------
sampler = TPESampler(seed=89)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = 10)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros
#    - Abrimos un run "padre" para agrupar toda la búsqueda
# ------------------------------------------------------------
with mlflow.start_run(run_name="Random Forest Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective_rf, n_trials=10)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    best_params["random_state"] = 89
    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "random_forest",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # Entrenar un modelo FINAL con los mejores hiperparámetros
    # --------------------------------------------------------
    model = RandomForestRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # Guardar artefactos adicionales (preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # --------------------------------------------------------
    # Guardar modelo final con signature
    # --------------------------------------------------------
    signature = infer_signature(X_val, y_pred)
    mlflow.sklearn.log_model(
        model,
        name="model",
        input_example=X_val[:5],
        signature=signature
    )

[I 2025-10-28 08:38:50,689] A new study created in memory with name: no-name-efaacc4e-7477-429a-bbb8-e8fdc0cfbaf8


2025/10/28 08:39:15 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 08:39:35,222] Trial 0 finished with value: 8.12809579814196 and parameters: {'n_estimators': 275, 'max_depth': 16, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False}. Best is trial 0 with value: 8.12809579814196.


🏃 View run bald-trout-228 at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/9ff06e07705e46d494b64d8f3fba437b
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


2025/10/28 08:39:49 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 08:39:52,884] Trial 1 finished with value: 8.78498393108378 and parameters: {'n_estimators': 95, 'max_depth': 23, 'min_samples_split': 19, 'min_samples_leaf': 4, 'max_features': 'log2', 'bootstrap': False}. Best is trial 0 with value: 8.12809579814196.


🏃 View run likeable-squid-971 at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/95dd9e5371b94afc9b2a75b868c0b357
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


...

🏃 View run caring-penguin-862 at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/a8598bca01784abc88cb9e8e87b7c3c7
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


2025/10/28 08:43:46 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 08:43:51,134] Trial 9 finished with value: 8.970877841342704 and parameters: {'n_estimators': 240, 'max_depth': 38, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_features': 'log2', 'bootstrap': True}. Best is trial 5 with value: 7.791217478989728.


🏃 View run traveling-lark-534 at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/40eec0f432a04c739c5af3057bb15e38
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


2025/10/28 08:44:14 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run Random Forest Hyperparameter Optimization (Optuna) at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551/runs/07107074ae4e4cc0ad7201b6b3dda81d
🧪 View experiment at: https://dbc-905d578f-b3ee.cloud.databricks.com/ml/experiments/86265904456551


## Registrar Modelo

In [119]:
model_name = "workspace.default.nyc-taxi-model"

In [113]:
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    order_by=["metrics.rmse ASC"],
    output_format="list"
)

# Obtener el mejor run
if len(runs) > 0:
    best_run = runs[0]
    print("🏆 Champion Run encontrado:")
    print(f"Run ID: {best_run.info.run_id}")
    print(f"RMSE: {best_run.data.metrics['rmse']}")
    print(f"Params: {best_run.data.params}")
else:
    print("⚠️ No se encontraron runs con métrica RMSE.")

🏆 Champion Run encontrado:
Run ID: 48e8894e2f3742929e9a40027f93fdad
RMSE: 5.437591039019735
Params: {'alpha': '0.9', 'ccp_alpha': '0.0', 'criterion': 'friedman_mse', 'init': 'None', 'learning_rate': '0.026706970597024548', 'loss': 'squared_error', 'max_depth': '7', 'max_features': 'None', 'max_leaf_nodes': 'None', 'min_impurity_decrease': '0.0', 'min_samples_leaf': '6', 'min_samples_split': '4', 'min_weight_fraction_leaf': '0.0', 'n_estimators': '406', 'n_iter_no_change': 'None', 'random_state': '89', 'subsample': '0.53280781534311', 'tol': '0.0001', 'validation_fraction': '0.1', 'verbose': '0', 'warm_start': 'False'}


In [69]:
run_id = best_run.info.run_id

In [70]:
result = mlflow.register_model(
    model_uri=f"runs:/{best_run.info.run_id}/model",
    name=model_name
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/28 08:44:30 WARNING mlflow.tracking._model_registry.fluent: Run with id 48e8894e2f3742929e9a40027f93fdad has no artifacts at artifact path 'model', registering model based on models:/m-c8fbdf448bda43d995b15d4e84cc7145 instead


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Created version '6' of model 'workspace.default.nyc-taxi-model'.


## Asignar alias "Challenger"

In [114]:
from mlflow import MlflowClient

client = MlflowClient()

In [72]:
model_version = result.version
new_alias = "Challenger"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

In [73]:
from datetime import datetime

date = datetime.today()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1761662675976, current_stage=None, deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description=('The model version 6 was transitioned to Challenger on 2025-10-28 '
 '08:44:43.303288'), last_updated_timestamp=1761662683770, metrics=[<Metric: dataset_digest='', dataset_name='', key='rmse', model_id='m-c8fbdf448bda43d995b15d4e84cc7145', run_id='48e8894e2f3742929e9a40027f93fdad', step=0, timestamp=1761662252093, value=5.437591039019735>,
 <Metric: dataset_digest='', dataset_name='', key='root_mean_squared_error_unknown_dataset', model_id='m-c8fbdf448bda43d995b15d4e84cc7145', run_id='48e8894e2f3742929e9a40027f93fdad', step=0, timestamp=1761662251584, value=5.437591039019735>,
 <Metric: dataset_digest='', dataset_name='', key='training_mean_absolute_error', model_id='m-c8fbdf448bda43d995b15d

## Predicciones

In [135]:
from sklearn.metrics import mean_squared_error

In [139]:
import mlflow.pyfunc

model_version_uri = f"models:/{model_name}@Champion"

champion_model = mlflow.pyfunc.load_model(model_version_uri)
y_pred_champion = champion_model.predict(X_test)


c:\Users\sarah\apps\sem_5\pcd\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [11:21:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


In [150]:
import mlflow.pyfunc

model_version_uri = f"models:/{model_name}@Challenger"

challenger_model = mlflow.pyfunc.load_model(model_version_uri)
y_pred_challenger = challenger_model.predict(X_test)

In [151]:
rmse_champion = np.sqrt(mean_squared_error(y_test, y_pred_champion))
print("RMSE Champion en X_test:", rmse_champion)

rmse_challenger = np.sqrt(mean_squared_error(y_test, y_pred_challenger))
print("RMSE Challenger en X_test:", rmse_challenger)

RMSE Champion en X_test: 6.383278117047989
RMSE Challenger en X_test: 6.002214055571562


Si se promovera el Challenger a Champion dado a que el rmse es menor por aprox. un 6%

![Pruebas](https://drive.google.com/uc?export=view&id=182pjlBuGfDjpfWzzVJTLitZovUvuFgIC)
